<a href="https://colab.research.google.com/github/Gayathricodes/Tamil-MuRIL/blob/main/TamilDataSet_MuRIL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow_hub import KerasLayer

In [ ]:
!gpustat

/bin/bash: line 1: gpustat: command not found


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install gpustat

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.1/98.1 kB 1.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 5.2 MB/s eta 0:00:00
  Created wheel for gpustat: filename=gpustat-1.1.1-py3-none-any.whl size=26535 sha256=a127fc28830e99cdea2446c1b7b5d0f7562896bec183151f361be105a6308558
  Stored in directory: /root/.cache/pip/wheels/ec/d7/80/a71ba3540900e1f276bcae685efd8e590c810d2108b95f1e47
Successfully built gpustat


In [ ]:
!pwd
import os
os.chdir('/content/drive/My Drive/MuRIL')
!pwd

/content
/content/drive/My Drive/MuRIL


In [ ]:
!pip install bert-for-tf2
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 685.5 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-py3-none-any.whl size=30509 sha256=e298edecc4120eca804f9306366b17c30dd98d911f3959c96c6b754e5c301ba4
  Stored in directory: /root/.cache/pip/wheels/d8/da/50/126d7b8416d9a0e6bf876935c2219a71e72a6529c25e150c56
  Created wheel for params-flow: filename=params_flow-0.8.2-py3-none-any.whl size=19454 sha256=69e60282ff3321cc8e720d11d6fed4ac7bf06a8f56f4a9545d58477b0837eb00
  Stored in directory: /root/.cache/pip/wheels/97/a8/d0/f7419404174976a2686bb98b5c30df01cc71445415f32db9e6
  Created wheel for py-params: filename=py_params-0.10.2-py3-none-any.whl size=7891 sha256=4dec38ff48b6f856d21050181d1ef67416596e633d7cedb1f832a862edc37d8f
  Stored in directory: /root/.cache/pip/wheels/69/c8/b3/92666cff9fb312bc3473eaa6b396695b89a7b3e31e9087

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import bert
from tensorflow.keras.models import  Model
from tqdm import tqdm
import numpy as np
import pandas as pd
from collections import namedtuple
from sklearn import preprocessing
from bert import bert_tokenization
print("TensorFlow Version:",tf.__version__)
print("Hub version: ",hub.__version__)

TensorFlow Version: 2.15.0
Hub version:  0.16.1


In [ ]:
# Load train and val datasets
df_train = pd.read_csv('tamilds_train_80.tsv', sep = "\t")
df_val = pd.read_csv('tamilds_val_20.tsv', sep = "\t")

In [ ]:
# Columns
df_train.columns

Index(['Message', 'Type'], dtype='object')

In [ ]:
# Prepare input text and one hot encoded labels for train and validation sets

unique_labels = list(np.unique(df_train["Type"]))

train_x = df_train["Message"].values
train_y = df_train["Type"].values

le = preprocessing.LabelEncoder()

train_y = le.fit_transform(train_y)
train_y = tf.keras.utils.to_categorical(train_y, num_classes=len(unique_labels), dtype='float32')

val_x = df_val["Message"].values
val_y = df_val["Type"].values

val_y = le.fit_transform(val_y)
val_y = tf.keras.utils.to_categorical(val_y, num_classes=len(unique_labels), dtype='float32')


print("number of unique labels", len(unique_labels))

number of unique labels 2


In [ ]:
# Check unique labels
unique_labels

['Ham', 'Spam']

In [ ]:
# Function to create input_ids
def get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens,)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

# Function to create attention masks
def get_masks(tokens, max_seq_length):
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))

# Function to create segment ids
def get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))

# Function to create input_ids, attention_masks, segment_ids for sample
def create_single_input(sentence,MAX_LEN, MAX_SEQ_LEN):

  stokens = tokenizer.tokenize(sentence)

  stokens = stokens[:MAX_LEN]

  stokens = ["[CLS]"] + stokens + ["[SEP]"]

  ids = get_ids(stokens, tokenizer, MAX_SEQ_LEN)
  masks = get_masks(stokens, MAX_SEQ_LEN)
  segments = get_segments(stokens, MAX_SEQ_LEN)

  return ids,masks,segments

def create_input_array(sentences, MAX_SEQ_LEN):

  input_ids, input_masks, input_segments = [], [], []

  for sentence in tqdm(sentences,position=0, leave=True):

    ids,masks,segments=create_single_input(sentence,MAX_SEQ_LEN-2, MAX_SEQ_LEN)

    input_ids.append(ids)
    input_masks.append(masks)
    input_segments.append(segments)

  return [np.asarray(input_ids, dtype=np.int32),
            np.asarray(input_masks, dtype=np.int32),
            np.asarray(input_segments, dtype=np.int32)]

In [ ]:
# MuRIL model layer
muril_layer = hub.KerasLayer("https://tfhub.dev/google/MuRIL/1", trainable=True)

# Create tokenizer
vocab_file = muril_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = muril_layer.resolved_object.do_lower_case.numpy()
tokenizer = bert_tokenization.FullTokenizer(vocab_file, do_lower_case)

In [ ]:
# Create input_ids, attention_masks, segment_ids for training and validation sets with max_seq_len as 128
max_seq_len = 128
train_x = create_input_array(train_x, max_seq_len)
val_x = create_input_array(val_x, max_seq_len)

100%|██████████| 235/235 [00:00<00:00, 2828.75it/s]


In [ ]:
# Define model function - compile and fit
def model_fit(train_x, train_y, val_x, val_y, max_seq_length, num_epochs, muril_layer):

  input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                       name="input_word_ids")
  input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                   name="input_mask")
  segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                    name="segment_ids")

  outputs = muril_layer(dict(input_word_ids = input_word_ids, input_mask = input_mask, input_type_ids = segment_ids))

  x = tf.keras.layers.Dropout(0.2)(outputs["pooled_output"]) # take pooled output layer
  final_output = tf.keras.layers.Dense(2, activation="softmax")(x)

  model = tf.keras.models.Model(
      inputs=[input_word_ids, input_mask, segment_ids], outputs=final_output)

  model.compile(loss='binary_crossentropy',
                  optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                  metrics=['accuracy'])
  model.fit(train_x, train_y, epochs = num_epochs, batch_size = 16, validation_data = (val_x, val_y), shuffle = False)

  return model

In [ ]:
# Set number of epochs
num_epochs = 10

# Get the model object
model = model_fit(train_x, train_y, val_x, val_y, max_seq_len, num_epochs, muril_layer)

Epoch 1/10
59/59 [==============================] - 58s 487ms/step - loss: 0.4121 - accuracy: 0.9011 - val_loss: 1.3265 - val_accuracy: 0.4298
Epoch 2/10
59/59 [==============================] - 27s 464ms/step - loss: 0.2966 - accuracy: 0.9170 - val_loss: 1.5752 - val_accuracy: 0.4298
Epoch 3/10
59/59 [==============================] - 28s 479ms/step - loss: 0.2993 - accuracy: 0.9170 - val_loss: 1.5927 - val_accuracy: 0.4298
Epoch 4/10
59/59 [==============================] - 28s 471ms/step - loss: 0.2989 - accuracy: 0.9170 - val_loss: 1.5930 - val_accuracy: 0.4298
Epoch 5/10
59/59 [==============================] - 28s 480ms/step - loss: 0.2978 - accuracy: 0.9170 - val_loss: 1.5666 - val_accuracy: 0.4298
Epoch 6/10
59/59 [==============================] - 28s 480ms/step - loss: 0.2955 - accuracy: 0.9170 - val_loss: 1.5633 - val_accuracy: 0.4298
Epoch 7/10
59/59 [==============================] - 28s 471ms/step - loss: 0.2950 - accuracy: 0.9170 - val_loss: 1.5644 - val_accuracy: 0.4298

In [ ]:
score = model.evaluate(val_x, val_y, verbose = 0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
import matplotlib.pyplot as plt


accuracy = accuracy_score(val_x, val_y)

# Precision
precision = precision_score(val_x, val_y)

# Recall
recall = recall_score(val_x, val_y)

# F1-score
f1 = f1_score(val_x, val_y)

# AUC-ROC
auc_roc = roc_auc_score(val_x, val_y)

# ROC Curve
fpr, tpr, thresholds = roc_curve(val_x, val_y)

# Plot ROC curve
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % auc_roc)
plt.plot([0, 1], [0, 1], 'k--')  # Random guess curve
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()
